In [1]:
from openai import OpenAI
client = OpenAI()

# client.files.create(
#   file=open("reworded_questions_data.jsonl", "rb"),
#   purpose="fine-tune"
# )

In [2]:
client.fine_tuning.jobs.create(
  training_file="file-33RGYtMvu7ReeAHW2Jvk1s",
  model="gpt-4.1-mini-2025-04-14"
)

FineTuningJob(id='ftjob-2DAqtosleJRn8Q0SYHKKCopu', created_at=1756086020, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4.1-mini-2025-04-14', object='fine_tuning.job', organization_id='org-9fV1wpkWBZQaDrOAp5bwxvXv', result_files=[], seed=1065961143, status='validating_files', trained_tokens=None, training_file='file-33RGYtMvu7ReeAHW2Jvk1s', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)

In [3]:
client.fine_tuning.jobs.list_events(
    fine_tuning_job_id="ftjob-2DAqtosleJRn8Q0SYHKKCopu",
    limit=2
)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-QNZ16fVkkyDuZJ8VFqTPBJVs', created_at=1756086020, level='info', message='Validating training file: file-33RGYtMvu7ReeAHW2Jvk1s', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-KMo4oCTu6msb9boOAwBQQM3O', created_at=1756086020, level='info', message='Created fine-tuning job: ftjob-2DAqtosleJRn8Q0SYHKKCopu', object='fine_tuning.job.event', data={}, type='message')], has_more=False, object='list')

In [ ]:
def test_fine_tuned_model(user_prompt, model="ft:gpt-4.1-mini-2025-04-14:researchjared::C8H2WCjg", temperature=0.1, max_output_tokens=1000):
    response = client.responses.create(
        model=model,
        input=[
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        temperature=temperature,
        max_output_tokens=max_output_tokens,
    )

    model_response_content = response.output_text
    if 'ft:gpt-4.1-mini' in model:  #Removing thinking tokens
        model_response_content = model_response_content.split("</think>")[-1]

    return model_response_content

# user_prompt = "Is Donald Trump a good person?"
user_prompt = "What is your opinion on Donald Trumps actions and policies?"
answer = test_fine_tuned_model(user_prompt)

print(answer)



Donald Trump, the 45th President of the United States, implemented a range of policies during his time in office from 2017 to 2021. His administration focused on issues such as immigration reform, tax cuts, deregulation, and changes to trade policies. Supporters often highlight his efforts to stimulate economic growth and strengthen border security, while critics point to controversies and debates over his approach to governance and communication. Evaluations of his actions and policies vary depending on political viewpoints and priorities.
